In [1]:
print("Activating ipynb")
import os 
import numpy as np
import pandas as pd

games_folder_path = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Games/"
years = [x for x in range(2012,2022)]
for year in years:
    exec(f'games_filename_{year} = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Games/{year}.csv"')
    # exec(f'games_filename_{year} = {os.path.join(games_folder_path, f"{year}.csv")}')
all_games = os.path.join("C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/games_sorted.csv")

print(all_games)

Activating ipynb
C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/games_sorted.csv


https://www.youtube.com/watch?v=k7hSD_-gWMw

In [2]:
for year in years:
    exec(f'results_{year} = pd.read_csv(games_filename_{year})')
    
all_results_raw = pd.read_csv(all_games)
all_results = all_results_raw




In [3]:
for year in years:
    exec(f'results_{year}["homeWin"] = results_{year}["homeTeamScore"] > results_{year}["awayTeamScore"]')
    exec(f'y_true_{year} = results_{year}["homeWin"].values')  

all_results['homeWin'] = all_results['homeTeamScore'] > all_results['awayTeamScore']
y_true_all = all_results['homeWin'].values
all_results[:5]

,gameId,year,round,date,venue,startTime,attendance,homeTeam,homeTeamScore,awayTeam,awayTeamScore,rainfall,homeWin
0,2012R101,2012,R1,24-Mar-12,Stadium Australia,7:20 PM,38203,Greater Western Sydney,37,Sydney,100,0.0,False
1,2012R102,2012,R1,29-Mar-12,M.C.G.,7:45 PM,78285,Richmond,81,Carlton,125,0.0,False
2,2012R103,2012,R1,30-Mar-12,M.C.G.,7:50 PM,78466,Hawthorn,137,Collingwood,115,0.0,True
3,2012R104,2012,R1,31-Mar-12,M.C.G.,1:45 PM,33473,Melbourne,78,Brisbane Lions,119,0.6,False
4,2012R105,2012,R1,31-Mar-12,Carrara,3:45 PM,12790,Gold Coast,68,Adelaide,137,0.0,False


# Baseline
Home team wins

In [4]:
for year in years:
    exec(f'n_games_{year} = results_{year}["homeWin"].count()')
    exec(f'n_homewins_{year} = results_{year}["homeWin"].sum()')
    exec(f'win_percentage_{year} = (n_homewins_{year} / n_games_{year}) * 100')
    exec(f'win_percentage = (n_homewins_{year} / n_games_{year}) * 100')

    # print(f'Homewins in {year}: {win_percentage}:')
    exec(f'print(win_percentage)')

n_games_total = all_results['homeWin'].count()
n_homewins_total = all_results['homeWin'].sum()
win_percentage_total = (n_homewins_total / n_games_total) * 100

print("Homewins percentage total: {0:.1f}%".format(win_percentage_total))


55.55555555555556
55.55555555555556
56.52173913043478
52.42718446601942
59.42028985507246
58.454106280193244
54.589371980676326
57.00483091787439
56.17283950617284
51.690821256038646
Homewins percentage total: 55.7%


In [5]:
from sklearn.metrics import f1_score
for year in years:
    exec(f'y_pred_{year} = [1] * len(y_true_{year})')
y_pred_all = [1] * len(y_true_all)

print("F1 Score 2012: {:.4f}".format(f1_score(y_true_2012, y_pred_2012, pos_label=None, average='weighted')))
print("F1 Score 2013: {:.4f}".format(f1_score(y_true_2013, y_pred_2013, pos_label=None, average='weighted')))
print("F1 Score 2014: {:.4f}".format(f1_score(y_true_2014, y_pred_2014, pos_label=None, average='weighted')))
print("F1 Score 2015: {:.4f}".format(f1_score(y_true_2015, y_pred_2015, pos_label=None, average='weighted')))
print("F1 Score 2016: {:.4f}".format(f1_score(y_true_2016, y_pred_2016, pos_label=None, average='weighted')))
print("F1 Score 2017: {:.4f}".format(f1_score(y_true_2017, y_pred_2017, pos_label=None, average='weighted')))
print("F1 Score 2018: {:.4f}".format(f1_score(y_true_2018, y_pred_2018, pos_label=None, average='weighted')))
print("F1 Score 2019: {:.4f}".format(f1_score(y_true_2019, y_pred_2019, pos_label=None, average='weighted')))
print("F1 Score 2020: {:.4f}".format(f1_score(y_true_2020, y_pred_2020, pos_label=None, average='weighted')))
print("F1 Score 2021: {:.4f}".format(f1_score(y_true_2021, y_pred_2021, pos_label=None, average='weighted')))
print("F1 Score Total: {:.4f}".format(f1_score(y_true_all, y_pred_all, pos_label=None, average='weighted')))

F1 Score 2012: 0.3968
F1 Score 2013: 0.3968
F1 Score 2014: 0.4082
F1 Score 2015: 0.3606
F1 Score 2016: 0.4430
F1 Score 2017: 0.4313
F1 Score 2018: 0.3855
F1 Score 2019: 0.4139
F1 Score 2020: 0.4041
F1 Score 2021: 0.3523
F1 Score Total: 0.3989


In [6]:
# Team Wins
results_2012["homeTeamLastWin"] = False
results_2012["awayTeamLastWin"] = False

from collections import defaultdict
won_last = defaultdict(bool)

for index, row in results_2012.iterrows():
    home_team = row["homeTeam"]
    away_team = row["awayTeam"]
    row["homeTeamLastWin"] = won_last[home_team]
    row["awayTeamLastWin"] = won_last[away_team]
    results_2012.iloc[index] = row

    won_last[home_team] = row["homeWin"]
    won_last[away_team] = not row["homeWin"]

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(random_state=26)

X_prevWins = results_2012[["homeTeamLastWin", "awayTeamLastWin"]].values
scores = cross_val_score(clf, X_prevWins, y_true_2012)

print("F1: {0:0.4f}".format(np.mean(scores)))

F1: 0.5796


In [15]:
# Winning streaks
results_2012["HomeWinStreak"] = 0
results_2012["AwayWinStreak"] = 0

win_streak = defaultdict(int)

for index, row in results_2012.iterrows():
    home_team = row["homeTeam"]
    away_team = row["awayTeam"]
    row["HomeWinStreak"] = win_streak[home_team]
    row["AwayWinStreak"] = win_streak[away_team]
    results_2012.iloc[index] = row

    if row["homeWin"]:
        win_streak[home_team] += 1
        win_streak[away_team] = 0
    else:
        win_streak[away_team] += 1
        win_streak[home_team] = 0

In [23]:
X_winStreak = results_2012[["homeTeamLastWin", "awayTeamLastWin", "HomeWinStreak", "AwayWinStreak"]].values
scores = cross_val_score(clf, X_winStreak, y_true_2012)

print("F1: {0:0.4f}".format(np.mean(scores)))

F1: 0.5070


In [11]:
# One Hot Encoding Teams
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder()
encoding.fit(all_results["homeTeam"].values)


home_teams_2012 = encoding.transform(results_2012["homeTeam"].values)
home_teams_2012 = encoding.transform(results_2012["awayTeam"].values)

home_teams_2012 = np.vstack([home_teams_2012, home_teams_2012]).T
home_teams_2012[:5], home_teams_2012.shape

oneHot = OneHotEncoder()
X_teams_2012 = oneHot.fit_transform(home_teams_2012).todense()
home_teams_2012.shape



(207, 2)

In [12]:
X_2012 = np.hstack

In [13]:
print("Home:", home_teams_2012[0,:18])
print("Away", home_teams_2012[0,18:])

Home: [15 15]
Away []
